In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
abdomen_test = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_test.csv')
abdomen_train = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_train.csv')
abdomen_val = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/abdomen_val.csv')

In [6]:
abdomen_test_sdp = abdomen_test[abdomen_test['Class'] == 2]
abdomen_train_sdp = abdomen_train[abdomen_train['Class'] == 2]
abdomen_val_sdp = abdomen_val[abdomen_val['Class'] == 2]

In [9]:
abdomen_val_sdp.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_val_sdp.csv', index=False)
abdomen_test_sdp.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_test_sdp.csv', index=False)
abdomen_train_sdp.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_train_sdp.csv', index=False)

In [2]:
abdomen_val_sdp = pd.read_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_val_sdp.csv')
abdomen_test_sdp = pd.read_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_test_sdp.csv')
abdomen_train_sdp = pd.read_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/abdomen_train_sdp.csv')

In [2]:
# for 7 class classification
train_set = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/train_set7.csv')
val_set = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/val_set7.csv')
test_set = pd.read_csv('/data/kpusteln/fetal/standard_plane/class_data/test_set7.csv')


In [ ]:
"""new_labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - abdomen non-standard plane
4 - abdomen standard plane
5 - femur non standard plane
6 - femur standard plane"""

In [3]:
train_set_stp = train_set[(train_set['Class'] == 2) | (train_set['Class'] == 4) | (train_set['Class'] == 6)]
val_set_stp = val_set[(val_set['Class'] == 2) | (val_set['Class'] == 4) | (val_set['Class'] == 6)]
test_set_stp = test_set[(test_set['Class'] == 2) | (test_set['Class'] == 4) | (test_set['Class'] == 6)]

In [4]:
head_biometry = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/head_biometry.csv')
femur_biometry = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/femur_biometry.csv')
abdomen_biometry = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/abdomen_biometry.csv')


In [5]:
head_biometry.rename(columns={'HC': 'measure', 'head_ps': 'ps'}, inplace=True)
femur_biometry.rename(columns={'FL': 'measure', 'femur_ps': 'ps'}, inplace=True)
abdomen_biometry.rename(columns={'AC': 'measure', 'abdomen_ps': 'ps'}, inplace=True)


In [6]:
biometry_merged = pd.concat([head_biometry, femur_biometry, abdomen_biometry], axis=0)

In [7]:
train_set_biometry = pd.merge(train_set_stp, biometry_merged, on='index', how='inner')
val_set_biometry = pd.merge(val_set_stp, biometry_merged, on='index', how='inner')
test_set_biometry = pd.merge(test_set_stp, biometry_merged, on='index', how='inner')

In [8]:
train_set_biometry

,index,Class_x,video_x,Class_y,video_y,measure,ps,frames_n
0,1_1_15,2,1_1,2,1_1,20.19,0.096716,68
1,1_1_16,2,1_1,2,1_1,20.19,0.096716,68
2,1_1_17,2,1_1,2,1_1,20.19,0.096716,68
3,1_1_18,2,1_1,2,1_1,20.19,0.096716,68
4,1_1_19,2,1_1,2,1_1,20.19,0.096716,68
...,...,...,...,...,...,...,...,...
38670,707_3_20,6,707_3,6,707_3,6.41,0.136138,27
38671,707_3_21,6,707_3,6,707_3,6.41,0.136138,27
38672,707_3_22,6,707_3,6,707_3,6.41,0.136138,27
38673,707_3_23,6,707_3,6,707_3,6.41,0.136138,27


In [9]:
train_set_biometry.drop(['video_y', 'Class_y'], axis=1, inplace=True)
val_set_biometry.drop(['video_y', 'Class_y'], axis=1, inplace=True)
test_set_biometry.drop(['video_y', 'Class_y'], axis=1, inplace=True)

In [10]:
train_set_biometry.rename(columns={'video_x': 'video', 'Class_x': 'Class'}, inplace=True)
val_set_biometry.rename(columns={'video_x': 'video', 'Class_x': 'Class'}, inplace=True)
test_set_biometry.rename(columns={'video_x': 'video', 'Class_x': 'Class'}, inplace=True)

In [25]:
train_set_biometry['measure_scaled'] = train_set_biometry['measure']/train_set_biometry['ps']
val_set_biometry['measure_scaled'] = val_set_biometry['measure']/val_set_biometry['ps']
test_set_biometry['measure_scaled'] = test_set_biometry['measure']/test_set_biometry['ps']

In [19]:
scaler = MinMaxScaler()
scaler.fit(train_set_biometry[['measure_scaled']])

MinMaxScaler()

In [29]:
train_set_biometry['measure_normalized'] = scaler.transform(train_set_biometry[['measure_scaled']])
val_set_biometry['measure_normalized'] = scaler.transform(val_set_biometry[['measure_scaled']])
test_set_biometry['measure_normalized'] = scaler.transform(test_set_biometry[['measure_scaled']])

In [52]:
train_set_biometry.drop('measure_scaled', axis=1, inplace=True)
val_set_biometry.drop('measure_scaled', axis=1, inplace=True)
test_set_biometry.drop('measure_scaled', axis=1, inplace=True)

In [53]:
train_set_biometry.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/train_set_biometry.csv', index=False)
val_set_biometry.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/val_set_biometry.csv', index=False)
test_set_biometry.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/test_set_biometry.csv', index=False)

In [55]:
val_set_biometry

,index,Class,video,measure,ps,frames_n,measure_normalized
0,19_1_75,2,19_1,25.51,0.148224,304,0.782152
1,19_1_76,2,19_1,25.51,0.148224,304,0.782152
2,19_1_86,2,19_1,25.51,0.148224,304,0.782152
3,19_1_92,2,19_1,25.51,0.148224,304,0.782152
4,19_1_93,2,19_1,25.51,0.148224,304,0.782152
...,...,...,...,...,...,...,...
2280,673_3_20,6,673_3,4.33,0.125731,26,0.117789
2281,673_3_23,6,673_3,4.33,0.125731,26,0.117789
2282,673_3_24,6,673_3,4.33,0.125731,26,0.117789
2283,673_3_26,6,673_3,4.33,0.125731,26,0.117789


In [39]:
joblib.dump(scaler, 'scaler_filename')

['scaler_filename']

In [48]:
scaler = joblib.load('/data/kpusteln/Fetal-RL/data_preparation/scripts/scaler_filename')

In [51]:
train_set_biometry

,index,Class,video,measure,ps,frames_n,measure_scaled,measure_normalized
0,1_1_15,2,1_1,20.19,0.096716,68,208.755532,0.959028
1,1_1_16,2,1_1,20.19,0.096716,68,208.755532,0.959028
2,1_1_17,2,1_1,20.19,0.096716,68,208.755532,0.959028
3,1_1_18,2,1_1,20.19,0.096716,68,208.755532,0.959028
4,1_1_19,2,1_1,20.19,0.096716,68,208.755532,0.959028
...,...,...,...,...,...,...,...,...
38670,707_3_20,6,707_3,6.41,0.136138,27,47.084576,0.178817
38671,707_3_21,6,707_3,6.41,0.136138,27,47.084576,0.178817
38672,707_3_22,6,707_3,6.41,0.136138,27,47.084576,0.178817
38673,707_3_23,6,707_3,6.41,0.136138,27,47.084576,0.178817


In [29]:
test_set_biometry = pd.read_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/test_set_biometry.csv')

In [31]:
test_set_head = test_set_biometry[test_set_biometry['Class'] == 2] #head
test_set_abdomen = test_set_biometry[test_set_biometry['Class'] == 4] #abdomen
test_set_femur = test_set_biometry[test_set_biometry['Class'] == 6] #femur

In [32]:
test_set_abdomen.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/test_set_abdomen.csv', index=False)
test_set_head.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/test_set_head.csv', index=False)
test_set_femur.to_csv('/data/kpusteln/fetal/standard_plane/biometry_subset_class/test_set_femur.csv', index=False)